In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

from subprocess import check_output

In [2]:
train = pd.read_csv("data/train_2016_v2.csv", parse_dates=["transactiondate"])
print(train.head())
print('---------------------')
print(train.shape)

   parcelid  logerror transactiondate
0  11016594    0.0276      2016-01-01
1  14366692   -0.1684      2016-01-01
2  12098116   -0.0040      2016-01-01
3  12643413    0.0218      2016-01-02
4  14432541   -0.0050      2016-01-02
---------------------
(90275, 3)


In [3]:
prop = pd.read_csv("data/properties_2016.csv")
print(prop.head())
print('---------------------')
print(prop.shape)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   parcelid  airconditioningtypeid  architecturalstyletypeid  basementsqft  \
0  10754147                    NaN                       NaN           NaN   
1  10759547                    NaN                       NaN           NaN   
2  10843547                    NaN                       NaN           NaN   
3  10859147                    NaN                       NaN           NaN   
4  10879947                    NaN                       NaN           NaN   

   bathroomcnt  bedroomcnt  buildingclasstypeid  buildingqualitytypeid  \
0          0.0         0.0                  NaN                    NaN   
1          0.0         0.0                  NaN                    NaN   
2          0.0         0.0                  NaN                    NaN   
3          0.0         0.0                  3.0                    7.0   
4          0.0         0.0                  4.0                    NaN   

   calculatedbathnbr  decktypeid         ...           numberofstories  \
0           

In [4]:
prop_des = prop.describe(include='all').transpose()
prop_des["missing_ratio"] = 1-(prop_des["count"]/prop.shape[0])
prop_des["dtype"] = prop.dtypes
prop_des.reset_index().tail()

,index,count,unique,top,freq,mean,std,min,25%,50%,75%,max,missing_ratio,dtype
53,landtaxvaluedollarcnt,2.91748e+06,NaN,NaN,NaN,252478,445013,1,74836,167042,306918,9.02462e+07,0.0226895,float64
54,taxamount,2.95397e+06,NaN,NaN,NaN,5377.61,9183.11,1.34,2461.07,3991.78,6201.01,3.45886e+06,0.0104683,float64
55,taxdelinquencyflag,56462,1,Y,56462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.981086,object
56,taxdelinquencyyear,56464,NaN,NaN,NaN,13.8924,2.58101,0,14,14,15,99,0.981085,float64
57,censustractandblock,2.91009e+06,NaN,NaN,NaN,6.04843e+13,3.24903e+11,-1,6.0374e+13,6.03757e+13,6.05904e+13,4.8303e+14,0.025166,float64


In [5]:
# taxdelinquency는 "Y"를 True로 바꾸어 줌 (다른 dummy 변수들은 1이나 True로 되어 있음)
prop['taxdelinquencyflag'] = prop['taxdelinquencyflag'].replace('Y',True)
prop['taxdelinquencyflag'] = prop['taxdelinquencyflag'].fillna(False)

# taxdelinquencyyear는 4자리의 년도로 format을 맞춰줌
prop["taxdelinquencyyear"] = prop["taxdelinquencyyear"].apply(lambda x: x + 2000 if x <= 17 else x + 1900)

# 위도 경도를 원래 단위로 맞춰줌
prop["latitude"] = prop["latitude"]/1000000
prop["longitude"] = prop["longitude"]/1000000

In [6]:
vars = prop_des.reset_index()["index"].tolist()

cat_vars = ['transactiondate', 'transaction_year', 'transaction_month',
            'rawcensustractandblock', 'censustractandblock', 'fips',
            'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 
            'architecturalstyletypeid', 'decktypeid', 'buildingclasstypeid', 'storytypeid',
            'typeconstructiontypeid', 'propertycountylandusecode', 'propertylandusetypeid',
            'propertyzoningdesc', 'taxdelinquencyflag', 'taxdelinquencyyear', 'assessmentyear',
            'heatingorsystemtypeid', 'airconditioningtypeid', 'hashottuborspa', 'fireplaceflag',
            'pooltypeid10', 'pooltypeid2', 'pooltypeid7']

num_vars = [i for i in vars if i not in cat_vars]

num_vars.remove('parcelid')

print("number of categorical variables:", len(cat_vars))
print("number of numerical variables:", len(num_vars))


number of categorical variables: 28
number of numerical variables: 32


In [7]:
def cat_to_str(column_name):
    prop[column_name] = prop[column_name].astype("str")
    prop[column_name][prop[column_name] == 'nan'] = np.nan
    
ls_c_to_s = ['architecturalstyletypeid', 'rawcensustractandblock', 'censustractandblock',
             'regionidcounty', 'fips', 'regionidcity', 'regionidcounty', 'regionidneighborhood',
             'decktypeid', 'buildingclasstypeid', 'storytypeid', 'typeconstructiontypeid',
             'propertylandusetypeid', 'heatingorsystemtypeid', 'airconditioningtypeid']

for var in ls_c_to_s:
    cat_to_str(var)
    
# dummy 변수를 True/False로 인코딩
def cat_to_bln(column_name):
    prop[column_name] = prop[column_name].replace(1,True)
    prop[column_name] = prop[column_name].fillna(False)
    
c_to_b = ['fireplaceflag', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'hashottuborspa']

for var in c_to_b:
    cat_to_bln(var)
    
# cencus와 rawcensus의 앞 네자리로 변수를 만들고 다른 변수들과 묶기
prop["census_1"] = prop["censustractandblock"].astype("str").apply(lambda x: x[:4])
prop["census_1"][prop["census_1"] == 'nan'] = np.nan
prop["raw_census_1"] = prop["rawcensustractandblock"].astype("str").apply(lambda x: x[:4])
prop["raw_census_1"][prop["raw_census_1"] == 'nan'] = np.nan

# censustractandblock & rawcensustractandblock의 5-10자리 부분: census tract → class가 3000개 이상이라 너무 많은 것으로 생각됨
prop["census_2"] = prop["censustractandblock"].astype("str").apply(lambda x: x[4:10])
prop["census_2"][prop["census_2"] == ""] = np.nan

prop["raw_census_2"] = prop["rawcensustractandblock"].astype("str").apply(lambda x: x[4:11])
prop["raw_census_2"][prop["raw_census_2"] == ""] = np.nan

# censustractandblock & rawcensustractandblock의 5-10자리 부분 중 첫째 자리로 카테고리화 (카테고리 수를 줄임)
prop["census_3"] = prop["censustractandblock"].astype("str").apply(lambda x: x[4:5])
prop["census_3"][prop["census_3"] == ''] = np.nan
prop["raw_census_3"] = prop["rawcensustractandblock"].astype("str").apply(lambda x: x[4:5])
prop["raw_census_3"][prop["raw_census_3"] == ''] = np.nan

# censustractandblock & rawcensustractandblock의 1-5자리로 카테고리화
prop["census_4"] = prop["censustractandblock"].astype("str").apply(lambda x: x[:5])
prop["census_4"][prop["census_4"] == 'nan'] = np.nan
prop["raw_census_4"] = prop["rawcensustractandblock"].astype("str").apply(lambda x: x[:5])
prop["raw_census_4"][prop["raw_census_4"] == 'nan'] = np.nan

### 입력오류라고 보고 NaN값 처리
prop["regionidzip"][prop["regionidzip"]>=100000] = np.nan

# 첫글자로 새로운 변수 만들기 (첫 글자로 카테고리화)
prop["propertyzoningdesc_1"] = prop["propertyzoningdesc"].astype("str").apply(lambda x: x[:1])
prop["propertyzoningdesc_1"][prop["propertyzoningdesc_1"] == "n"] = np.nan

# 세번째 글자로 새로운 변수 만들기
prop["propertyzoningdesc_3"] = prop["propertyzoningdesc"].astype("str").apply(
    lambda x: x.replace("&", "").replace("-", "").replace(" ", ""))
prop["propertyzoningdesc_3"] = prop["propertyzoningdesc_3"].apply(lambda x: x[2:3])
prop["propertyzoningdesc_3"][prop["propertyzoningdesc_3"] == "n"] = np.nan
prop["propertyzoningdesc_3"][prop["propertyzoningdesc_3"] == ""] = np.nan

prop["taxdelinquencyyear_past"] = 2016 - prop["taxdelinquencyyear"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: S

In [8]:
vars = prop_des.reset_index()["index"].tolist()
vars.append("taxdelinquencyyear_past")

cat_vars = ['fips', 'rawcensustractandblock', 'censustractandblock', "raw_census_1", "census_1",
            "raw_census_2", "census_2", "raw_census_3", "census_3", "raw_census_4", "census_4",
            'regionidcounty', 'regionidcity', 'regionidzip', 'regionidneighborhood', 'architecturalstyletypeid',
            'decktypeid', 'buildingqualitytypeid', 'buildingclasstypeid', 
            'storytypeid', 'typeconstructiontypeid', 'propertycountylandusecode', 'propertylandusetypeid',
            'propertyzoningdesc', "propertyzoningdesc_1", "propertyzoningdesc_3",
            'taxdelinquencyflag', 'taxdelinquencyyear', 'assessmentyear',
            'heatingorsystemtypeid', 'airconditioningtypeid', 'hashottuborspa', 'fireplaceflag',
            'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'poolcnt']

num_vars = [i for i in vars if i not in cat_vars]

num_vars.remove('parcelid')

print("number of categorical variables:", len(cat_vars))
print("number of numerical variables:", len(num_vars))

number of categorical variables: 37
number of numerical variables: 31


In [9]:
def num_imputation(df):
    for var in num_vars:
        if var in ["garagecarcnt", "numberofstories"]:
            df[var] = df[var].fillna(1)
        elif var in ["buildingqualitytypeid", "roomcnt", "bedroomcnt", "unitcnt", 'bathroomcnt', 
                     'calculatedbathnbr', 'fullbathcnt', 'threequarterbathnbr', 'yearbuilt','finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
                     'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
                     'finishedsquarefeet50', 'finishedsquarefeet6', 'garagetotalsqft']:
            a = df[var].mode()
            df[var] = df[var].fillna(int(a[0]))
        elif var in ['lotsizesquarefeet',
                     'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'taxamount',
                     'latitude', 'longitude']:
            df[var] = df[var].fillna(df[var].mean())
        else:    # ['poolsizesum', 'fireplacecnt', 'yardbuildingsqft17', 'yardbuildingsqft26', "poolsizesum","taxdelinquencyyear_past"]
            df[var] = df[var].fillna(0)

In [10]:
num_imputation(prop)

In [11]:
def cat_imputation(df):
    for var in cat_vars:                  
        if var in ['taxdelinquencyyear', 'assessmentyear']:
            df[var] = df[var].fillna(0)
        elif var == "poolcnt":
            df[var] = df[var].replace(1, True)
            df[var] = df[var].fillna(False)            
        elif var == "decktypeid":
            df[var] = df[var].replace('66.0', True)
            df[var] = df[var].fillna(False)         
        else:
            a = df[var].mode()
            df[var] = df[var].fillna(a[0])

In [12]:
cat_imputation(prop)

In [13]:
prop_des = prop.describe(include='all').transpose()
prop_des["missing_ratio"] = 1-(prop_des["count"]/prop.shape[0])
prop_des["dtype"] = prop.dtypes
prop_missing = pd.DataFrame(prop_des["missing_ratio"].sort_values(ascending=False))
prop_missing.reset_index()

,index,missing_ratio
0,taxdelinquencyyear_past,0
1,latitude,0
2,fireplacecnt,0
3,fullbathcnt,0
4,garagecarcnt,0
5,garagetotalsqft,0
6,hashottuborspa,0
7,heatingorsystemtypeid,0
8,longitude,0
9,propertylandusetypeid,0


#### * LabelEncoding

In [14]:
for c in prop[['fips', 'rawcensustractandblock', 'censustractandblock', "raw_census_1", "census_1",
            "raw_census_2", "census_2", "raw_census_3", "census_3", "raw_census_4", "census_4",
            'regionidcounty', 'regionidcity', 'regionidzip', 'regionidneighborhood', 'architecturalstyletypeid',
            'decktypeid', 'buildingqualitytypeid', 'buildingclasstypeid', 
            'storytypeid', 'typeconstructiontypeid', 'propertycountylandusecode', 'propertylandusetypeid',
            'propertyzoningdesc', "propertyzoningdesc_1", "propertyzoningdesc_3",
            'taxdelinquencyflag', 'taxdelinquencyyear', 'assessmentyear',
            'heatingorsystemtypeid', 'airconditioningtypeid', 'hashottuborspa', 'fireplaceflag',
            'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'poolcnt']]:
    label = LabelEncoder()
    label.fit(list(prop[c].values))
    prop[c] = label.transform(list(prop[c].values))

In [15]:
prop

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,raw_census_1,census_2,raw_census_2,census_3,raw_census_3,census_4,raw_census_4,propertyzoningdesc_1,propertyzoningdesc_3,taxdelinquencyyear_past
0,10754147,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,2891,0,8,11,7,22,26,0.0
1,10759547,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,2887,0,8,11,7,22,10,0.0
2,10843547,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,2883,0,7,11,6,22,12,0.0
3,10859147,0,6,0.0,0.0,0.0,2,6,2.0,0,...,0,355,1072,0,1,11,0,22,12,0.0
4,10879947,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,915,0,1,11,0,22,21,0.0
5,10898347,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,946,0,1,11,0,22,12,0.0
6,10933547,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,1088,0,1,11,0,22,12,0.0
7,10940747,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,1780,0,3,11,2,12,12,0.0
8,10954547,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,1769,0,3,11,2,12,26,0.0
9,10976347,0,6,0.0,0.0,0.0,2,6,2.0,0,...,0,355,1794,0,3,11,2,28,12,0.0


In [16]:
train['transaction_month'] = pd.DatetimeIndex(train['transactiondate']).month

In [17]:
test = prop.drop(['parcelid'], axis=1)

In [18]:
test

,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,raw_census_1,census_2,raw_census_2,census_3,raw_census_3,census_4,raw_census_4,propertyzoningdesc_1,propertyzoningdesc_3,taxdelinquencyyear_past
0,0,6,0.0,0.0,0.0,3,6,2.0,0,1252.0,...,0,355,2891,0,8,11,7,22,26,0.0
1,0,6,0.0,0.0,0.0,3,6,2.0,0,1252.0,...,0,355,2887,0,8,11,7,22,10,0.0
2,0,6,0.0,0.0,0.0,3,6,2.0,0,1252.0,...,0,355,2883,0,7,11,6,22,12,0.0
3,0,6,0.0,0.0,0.0,2,6,2.0,0,1252.0,...,0,355,1072,0,1,11,0,22,12,0.0
4,0,6,0.0,0.0,0.0,3,6,2.0,0,1252.0,...,0,355,915,0,1,11,0,22,21,0.0
5,0,6,0.0,0.0,0.0,3,6,2.0,0,1252.0,...,0,355,946,0,1,11,0,22,12,0.0
6,0,6,0.0,0.0,0.0,3,6,2.0,0,1252.0,...,0,355,1088,0,1,11,0,22,12,0.0
7,0,6,0.0,0.0,0.0,3,6,2.0,0,1252.0,...,0,355,1780,0,3,11,2,12,12,0.0
8,0,6,0.0,0.0,0.0,3,6,2.0,0,1252.0,...,0,355,1769,0,3,11,2,12,26,0.0
9,0,6,0.0,0.0,0.0,2,6,2.0,0,1252.0,...,0,355,1794,0,3,11,2,28,12,0.0


In [19]:
train = pd.merge(train, prop, on='parcelid', how='left')
print(train.head())
print('---------------------')
print(train.shape)

   parcelid  logerror transactiondate  transaction_month  \
0  11016594    0.0276      2016-01-01                  1   
1  14366692   -0.1684      2016-01-01                  1   
2  12098116   -0.0040      2016-01-01                  1   
3  12643413    0.0218      2016-01-02                  1   
4  14432541   -0.0050      2016-01-02                  1   

   airconditioningtypeid  architecturalstyletypeid  basementsqft  bathroomcnt  \
0                      0                         6           0.0          2.0   
1                      0                         6           0.0          3.5   
2                      0                         6           0.0          3.0   
3                      0                         6           0.0          2.0   
4                      0                         6           0.0          2.5   

   bedroomcnt  buildingclasstypeid           ...             raw_census_1  \
0         3.0                    3           ...                        0  

In [20]:
log_errors = train['logerror']
train = train[train.logerror < np.percentile(log_errors, 99.5)]
train = train[train.logerror > np.percentile(log_errors, 0.5)]

print('upper limit: ', np.percentile(log_errors, 99.5))
print('lower limit: ', np.percentile(log_errors, 0.5))

upper limit:  0.7135300000000047
lower limit:  -0.5042


In [21]:
x_train = train.drop(['parcelid', 'logerror','transactiondate'], axis=1)
y_train = train['logerror']

In [22]:
params = {
    'eta': 0.02,
    'max_depth': 1,
    'subsample': 0.90,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': np.mean(y_train),
    'silent': 1
}

In [23]:
dtrain = xgb.DMatrix(x_train, y_train)

In [24]:
dtrain

In [25]:
cv_result = xgb.cv(params, 
                   dtrain, 
                   nfold=5,
                   num_boost_round=200,
                   early_stopping_rounds=5,
                   verbose_eval=10, 
                   show_stdv=False
                  )
num_boost_rounds = len(cv_result)
print(num_boost_rounds)

[0]	train-mae:0.0577206	test-mae:0.0577208
[10]	train-mae:0.0576932	test-mae:0.0576946
[20]	train-mae:0.05766	test-mae:0.0576634
[30]	train-mae:0.0576296	test-mae:0.0576342
[40]	train-mae:0.0576086	test-mae:0.0576146
[50]	train-mae:0.05759	test-mae:0.057597
[60]	train-mae:0.0575724	test-mae:0.05758
[70]	train-mae:0.057559	test-mae:0.0575678
[80]	train-mae:0.0575462	test-mae:0.0575558
[90]	train-mae:0.0575334	test-mae:0.0575436
[100]	train-mae:0.0575222	test-mae:0.0575342
[110]	train-mae:0.0575124	test-mae:0.0575254
[120]	train-mae:0.0575032	test-mae:0.057517
[130]	train-mae:0.0574924	test-mae:0.0575082
[140]	train-mae:0.0574852	test-mae:0.0575018
[150]	train-mae:0.0574784	test-mae:0.057496
[160]	train-mae:0.0574706	test-mae:0.05749
[170]	train-mae:0.0574612	test-mae:0.057483
[180]	train-mae:0.0574552	test-mae:0.0574784
[190]	train-mae:0.0574496	test-mae:0.057474
[199]	train-mae:0.0574454	test-mae:0.0574712
200


In [26]:
mdl = xgb.train(params, dtrain, num_boost_round=num_boost_rounds)

In [27]:
sub = pd.read_csv('data/sample_submission.csv')

In [28]:
oct_test = test.copy()

In [29]:
oct_test['transaction_month'] = np.repeat(10, oct_test.shape[0])

In [30]:
def reorder(df):
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df.loc[:, cols]
    return df

In [31]:
oct_test = reorder(oct_test)

In [32]:
oct_test

,transaction_month,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,raw_census_1,census_2,raw_census_2,census_3,raw_census_3,census_4,raw_census_4,propertyzoningdesc_1,propertyzoningdesc_3,taxdelinquencyyear_past
0,10,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,2891,0,8,11,7,22,26,0.0
1,10,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,2887,0,8,11,7,22,10,0.0
2,10,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,2883,0,7,11,6,22,12,0.0
3,10,0,6,0.0,0.0,0.0,2,6,2.0,0,...,0,355,1072,0,1,11,0,22,12,0.0
4,10,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,915,0,1,11,0,22,21,0.0
5,10,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,946,0,1,11,0,22,12,0.0
6,10,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,1088,0,1,11,0,22,12,0.0
7,10,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,1780,0,3,11,2,12,12,0.0
8,10,0,6,0.0,0.0,0.0,3,6,2.0,0,...,0,355,1769,0,3,11,2,12,26,0.0
9,10,0,6,0.0,0.0,0.0,2,6,2.0,0,...,0,355,1794,0,3,11,2,28,12,0.0


In [33]:
d_oct_test = xgb.DMatrix(oct_test)

In [34]:
d_oct_test

In [35]:
oct_pred = mdl.predict(d_oct_test)

In [36]:
ss = pd.read_csv('data/sample_submission.csv')

In [37]:
sub_oct_pred = []
sub_nov_pred = []
sub_dec_pred = []

for i,predict in enumerate(oct_pred):
    sub_oct_pred.append(str(round(predict,4)))
sub_oct_pred=np.array(sub_oct_pred)

In [38]:
nov_test = test.copy()

In [39]:
nov_test['transaction_month'] = np.repeat(11, nov_test.shape[0])

In [40]:
nov_test = reorder(nov_test)

In [41]:
d_nov_test = xgb.DMatrix(nov_test)

In [42]:
nov_pred = mdl.predict(d_nov_test)

In [43]:
for i,predict in enumerate(nov_pred):
    sub_nov_pred.append(str(round(predict,4)))
sub_nov_pred=np.array(sub_nov_pred)

In [44]:
dec_test = test.copy()

In [45]:
dec_test['transaction_month'] = np.repeat(12, dec_test.shape[0])

In [46]:
dec_test = reorder(dec_test)

In [47]:
d_dec_test = xgb.DMatrix(dec_test)

In [48]:
dec_pred = mdl.predict(d_dec_test)

In [49]:
for i,predict in enumerate(dec_pred):
    sub_dec_pred.append(str(round(predict,4)))
sub_dec_pred=np.array(sub_dec_pred)

In [50]:
sub = pd.DataFrame({'ParcelId': prop['parcelid'].astype(np.int32), '201610':sub_oct_pred, '201611': sub_nov_pred, '201612':sub_dec_pred})

In [51]:
sub

,ParcelId,201610,201611,201612
0,10754147,0.0073,0.0073,0.0073
1,10759547,0.0073,0.0073,0.0073
2,10843547,0.0189,0.0189,0.0189
3,10859147,0.0111,0.0111,0.0111
4,10879947,0.0073,0.0073,0.0073
5,10898347,0.0077,0.0077,0.0077
6,10933547,0.0062,0.0062,0.0062
7,10940747,0.01,0.01,0.01
8,10954547,0.0073,0.0073,0.0073
9,10976347,0.01,0.01,0.01


In [52]:
train = pd.read_csv("data/train_2017.csv", parse_dates=["transactiondate"])
print(train.head())
print('---------------------')
print(train.shape)

   parcelid  logerror transactiondate
0  14297519  0.025595      2017-01-01
1  17052889  0.055619      2017-01-01
2  14186244  0.005383      2017-01-01
3  12177905 -0.103410      2017-01-01
4  10887214  0.006940      2017-01-01
---------------------
(77613, 3)


In [53]:
prop = pd.read_csv("data/properties_2017.csv")
print(prop.head())
print('---------------------')
print(prop.shape)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   parcelid  airconditioningtypeid  architecturalstyletypeid  basementsqft  \
0  10754147                    NaN                       NaN           NaN   
1  10759547                    NaN                       NaN           NaN   
2  10843547                    NaN                       NaN           NaN   
3  10859147                    NaN                       NaN           NaN   
4  10879947                    NaN                       NaN           NaN   

   bathroomcnt  bedroomcnt  buildingclasstypeid  buildingqualitytypeid  \
0          0.0         0.0                  NaN                    NaN   
1          0.0         0.0                  NaN                    NaN   
2          0.0         0.0                  5.0                    NaN   
3          0.0         0.0                  3.0                    6.0   
4          0.0         0.0                  4.0                    NaN   

   calculatedbathnbr  decktypeid         ...           numberofstories  \
0           

In [54]:
prop_des = prop.describe(include='all').transpose()
prop_des["missing_ratio"] = 1-(prop_des["count"]/prop.shape[0])
prop_des["dtype"] = prop.dtypes
prop_des.reset_index().tail()

,index,count,unique,top,freq,mean,std,min,25%,50%,75%,max,missing_ratio,dtype
53,landtaxvaluedollarcnt,2.92529e+06,NaN,NaN,NaN,268456,486510,1,79700,176619,326100,9.40111e+07,0.0200743,float64
54,taxamount,2.96246e+06,NaN,NaN,NaN,5408.95,9675.57,0.24,2468.62,4007.62,6230.5,3.82318e+06,0.00762156,float64
55,taxdelinquencyflag,56515,1,Y,56515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.981068,object
56,taxdelinquencyyear,56517,NaN,NaN,NaN,13.8917,2.56217,0,14,14,15,99,0.981068,float64
57,censustractandblock,2.91023e+06,NaN,NaN,NaN,6.04843e+13,3.24913e+11,-1,6.0374e+13,6.03757e+13,6.05904e+13,4.8303e+14,0.0251188,float64


In [55]:
# taxdelinquency는 "Y"를 True로 바꾸어 줌 (다른 dummy 변수들은 1이나 True로 되어 있음)
prop['taxdelinquencyflag'] = prop['taxdelinquencyflag'].replace('Y',True)
prop['taxdelinquencyflag'] = prop['taxdelinquencyflag'].fillna(False)

# taxdelinquencyyear는 4자리의 년도로 format을 맞춰줌
prop["taxdelinquencyyear"] = prop["taxdelinquencyyear"].apply(lambda x: x + 2000 if x <= 17 else x + 1900)

# 위도 경도를 원래 단위로 맞춰줌
prop["latitude"] = prop["latitude"]/1000000
prop["longitude"] = prop["longitude"]/1000000

In [56]:
vars = prop_des.reset_index()["index"].tolist()

cat_vars = ['transactiondate', 'transaction_year', 'transaction_month',
            'rawcensustractandblock', 'censustractandblock', 'fips',
            'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 
            'architecturalstyletypeid', 'decktypeid', 'buildingclasstypeid', 'storytypeid',
            'typeconstructiontypeid', 'propertycountylandusecode', 'propertylandusetypeid',
            'propertyzoningdesc', 'taxdelinquencyflag', 'taxdelinquencyyear', 'assessmentyear',
            'heatingorsystemtypeid', 'airconditioningtypeid', 'hashottuborspa', 'fireplaceflag',
            'pooltypeid10', 'pooltypeid2', 'pooltypeid7']

num_vars = [i for i in vars if i not in cat_vars]

num_vars.remove('parcelid')

print("number of categorical variables:", len(cat_vars))
print("number of numerical variables:", len(num_vars))


number of categorical variables: 28
number of numerical variables: 32


In [57]:
def cat_to_str(column_name):
    prop[column_name] = prop[column_name].astype("str")
    prop[column_name][prop[column_name] == 'nan'] = np.nan
    
ls_c_to_s = ['architecturalstyletypeid', 'rawcensustractandblock', 'censustractandblock',
             'regionidcounty', 'fips', 'regionidcity', 'regionidcounty', 'regionidneighborhood',
             'decktypeid', 'buildingclasstypeid', 'storytypeid', 'typeconstructiontypeid',
             'propertylandusetypeid', 'heatingorsystemtypeid', 'airconditioningtypeid']

for var in ls_c_to_s:
    cat_to_str(var)
    
# dummy 변수를 True/False로 인코딩
def cat_to_bln(column_name):
    prop[column_name] = prop[column_name].replace(1,True)
    prop[column_name] = prop[column_name].fillna(False)
    
c_to_b = ['fireplaceflag', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'hashottuborspa']

for var in c_to_b:
    cat_to_bln(var)
    
# cencus와 rawcensus의 앞 네자리로 변수를 만들고 다른 변수들과 묶기
prop["census_1"] = prop["censustractandblock"].astype("str").apply(lambda x: x[:4])
prop["census_1"][prop["census_1"] == 'nan'] = np.nan
prop["raw_census_1"] = prop["rawcensustractandblock"].astype("str").apply(lambda x: x[:4])
prop["raw_census_1"][prop["raw_census_1"] == 'nan'] = np.nan

# censustractandblock & rawcensustractandblock의 5-10자리 부분: census tract → class가 3000개 이상이라 너무 많은 것으로 생각됨
prop["census_2"] = prop["censustractandblock"].astype("str").apply(lambda x: x[4:10])
prop["census_2"][prop["census_2"] == ""] = np.nan

prop["raw_census_2"] = prop["rawcensustractandblock"].astype("str").apply(lambda x: x[4:11])
prop["raw_census_2"][prop["raw_census_2"] == ""] = np.nan

# censustractandblock & rawcensustractandblock의 5-10자리 부분 중 첫째 자리로 카테고리화 (카테고리 수를 줄임)
prop["census_3"] = prop["censustractandblock"].astype("str").apply(lambda x: x[4:5])
prop["census_3"][prop["census_3"] == ''] = np.nan
prop["raw_census_3"] = prop["rawcensustractandblock"].astype("str").apply(lambda x: x[4:5])
prop["raw_census_3"][prop["raw_census_3"] == ''] = np.nan

# censustractandblock & rawcensustractandblock의 1-5자리로 카테고리화
prop["census_4"] = prop["censustractandblock"].astype("str").apply(lambda x: x[:5])
prop["census_4"][prop["census_4"] == 'nan'] = np.nan
prop["raw_census_4"] = prop["rawcensustractandblock"].astype("str").apply(lambda x: x[:5])
prop["raw_census_4"][prop["raw_census_4"] == 'nan'] = np.nan

### 입력오류라고 보고 NaN값 처리
prop["regionidzip"][prop["regionidzip"]>=100000] = np.nan

# 첫글자로 새로운 변수 만들기 (첫 글자로 카테고리화)
prop["propertyzoningdesc_1"] = prop["propertyzoningdesc"].astype("str").apply(lambda x: x[:1])
prop["propertyzoningdesc_1"][prop["propertyzoningdesc_1"] == "n"] = np.nan

# 세번째 글자로 새로운 변수 만들기
prop["propertyzoningdesc_3"] = prop["propertyzoningdesc"].astype("str").apply(
    lambda x: x.replace("&", "").replace("-", "").replace(" ", ""))
prop["propertyzoningdesc_3"] = prop["propertyzoningdesc_3"].apply(lambda x: x[2:3])
prop["propertyzoningdesc_3"][prop["propertyzoningdesc_3"] == "n"] = np.nan
prop["propertyzoningdesc_3"][prop["propertyzoningdesc_3"] == ""] = np.nan

prop["taxdelinquencyyear_past"] = 2016 - prop["taxdelinquencyyear"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: S

In [58]:
vars = prop_des.reset_index()["index"].tolist()
vars.append("taxdelinquencyyear_past")

cat_vars = ['fips', 'rawcensustractandblock', 'censustractandblock', "raw_census_1", "census_1",
            "raw_census_2", "census_2", "raw_census_3", "census_3", "raw_census_4", "census_4",
            'regionidcounty', 'regionidcity', 'regionidzip', 'regionidneighborhood', 'architecturalstyletypeid',
            'decktypeid', 'buildingqualitytypeid', 'buildingclasstypeid', 
            'storytypeid', 'typeconstructiontypeid', 'propertycountylandusecode', 'propertylandusetypeid',
            'propertyzoningdesc', "propertyzoningdesc_1", "propertyzoningdesc_3",
            'taxdelinquencyflag', 'taxdelinquencyyear', 'assessmentyear',
            'heatingorsystemtypeid', 'airconditioningtypeid', 'hashottuborspa', 'fireplaceflag',
            'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'poolcnt']

num_vars = [i for i in vars if i not in cat_vars]

num_vars.remove('parcelid')

print("number of categorical variables:", len(cat_vars))
print("number of numerical variables:", len(num_vars))

number of categorical variables: 37
number of numerical variables: 31


In [59]:
def num_imputation(df):
    for var in num_vars:
        if var in ["garagecarcnt", "numberofstories"]:
            df[var] = df[var].fillna(1)
        elif var in ["buildingqualitytypeid", "roomcnt", "bedroomcnt", "unitcnt", 'bathroomcnt', 
                     'calculatedbathnbr', 'fullbathcnt', 'threequarterbathnbr', 'yearbuilt','finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
                     'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
                     'finishedsquarefeet50', 'finishedsquarefeet6', 'garagetotalsqft']:
            a = df[var].mode()
            df[var] = df[var].fillna(int(a[0]))
        elif var in ['lotsizesquarefeet',
                     'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'taxamount',
                     'latitude', 'longitude']:
            df[var] = df[var].fillna(df[var].mean())
        else:    # ['poolsizesum', 'fireplacecnt', 'yardbuildingsqft17', 'yardbuildingsqft26', "poolsizesum","taxdelinquencyyear_past"]
            df[var] = df[var].fillna(0)

In [60]:
num_imputation(prop)

In [61]:
def cat_imputation(df):
    for var in cat_vars:                  
        if var in ['taxdelinquencyyear', 'assessmentyear']:
            df[var] = df[var].fillna(0)
        elif var == "poolcnt":
            df[var] = df[var].replace(1, True)
            df[var] = df[var].fillna(False)            
        elif var == "decktypeid":
            df[var] = df[var].replace('66.0', True)
            df[var] = df[var].fillna(False)         
        else:
            a = df[var].mode()
            df[var] = df[var].fillna(a[0])

In [62]:
cat_imputation(prop)

In [63]:
prop_des = prop.describe(include='all').transpose()
prop_des["missing_ratio"] = 1-(prop_des["count"]/prop.shape[0])
prop_des["dtype"] = prop.dtypes
prop_missing = pd.DataFrame(prop_des["missing_ratio"].sort_values(ascending=False))
prop_missing.reset_index()

,index,missing_ratio
0,taxdelinquencyyear_past,0
1,latitude,0
2,fireplacecnt,0
3,fullbathcnt,0
4,garagecarcnt,0
5,garagetotalsqft,0
6,hashottuborspa,0
7,heatingorsystemtypeid,0
8,longitude,0
9,propertylandusetypeid,0


#### * LabelEncoding

In [64]:
for c in prop[['fips', 'rawcensustractandblock', 'censustractandblock', "raw_census_1", "census_1",
            "raw_census_2", "census_2", "raw_census_3", "census_3", "raw_census_4", "census_4",
            'regionidcounty', 'regionidcity', 'regionidzip', 'regionidneighborhood', 'architecturalstyletypeid',
            'decktypeid', 'buildingqualitytypeid', 'buildingclasstypeid', 
            'storytypeid', 'typeconstructiontypeid', 'propertycountylandusecode', 'propertylandusetypeid',
            'propertyzoningdesc', "propertyzoningdesc_1", "propertyzoningdesc_3",
            'taxdelinquencyflag', 'taxdelinquencyyear', 'assessmentyear',
            'heatingorsystemtypeid', 'airconditioningtypeid', 'hashottuborspa', 'fireplaceflag',
            'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'poolcnt']]:
    label = LabelEncoder()
    label.fit(list(prop[c].values))
    prop[c] = label.transform(list(prop[c].values))

In [65]:
prop

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,raw_census_1,census_2,raw_census_2,census_3,raw_census_3,census_4,raw_census_4,propertyzoningdesc_1,propertyzoningdesc_3,taxdelinquencyyear_past
0,10754147,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,2892,0,8,11,7,22,26,0.0
1,10759547,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,2888,0,8,11,7,22,10,0.0
2,10843547,0,6,0.0,0.0,0.0,4,5,2.0,0,...,0,356,2884,0,7,11,6,22,12,0.0
3,10859147,0,6,0.0,0.0,0.0,2,5,2.0,0,...,0,356,1073,0,1,11,0,22,12,0.0
4,10879947,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,916,0,1,11,0,22,21,0.0
5,10898347,0,6,0.0,0.0,0.0,3,3,2.0,0,...,0,356,947,0,1,11,0,22,12,0.0
6,10933547,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,1089,0,1,11,0,22,12,0.0
7,10940747,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,1781,0,3,11,2,12,12,0.0
8,10954547,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,1770,0,3,11,2,12,26,0.0
9,10976347,0,6,0.0,0.0,0.0,2,3,2.0,0,...,0,356,1795,0,3,11,2,28,12,0.0


In [66]:
train['transaction_month'] = pd.DatetimeIndex(train['transactiondate']).month

In [67]:
test = prop.drop(['parcelid'], axis=1)

In [68]:
test

,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,raw_census_1,census_2,raw_census_2,census_3,raw_census_3,census_4,raw_census_4,propertyzoningdesc_1,propertyzoningdesc_3,taxdelinquencyyear_past
0,0,6,0.0,0.0,0.0,3,5,2.0,0,1252.0,...,0,356,2892,0,8,11,7,22,26,0.0
1,0,6,0.0,0.0,0.0,3,5,2.0,0,1252.0,...,0,356,2888,0,8,11,7,22,10,0.0
2,0,6,0.0,0.0,0.0,4,5,2.0,0,1252.0,...,0,356,2884,0,7,11,6,22,12,0.0
3,0,6,0.0,0.0,0.0,2,5,2.0,0,1252.0,...,0,356,1073,0,1,11,0,22,12,0.0
4,0,6,0.0,0.0,0.0,3,5,2.0,0,1252.0,...,0,356,916,0,1,11,0,22,21,0.0
5,0,6,0.0,0.0,0.0,3,3,2.0,0,1252.0,...,0,356,947,0,1,11,0,22,12,0.0
6,0,6,0.0,0.0,0.0,3,5,2.0,0,1252.0,...,0,356,1089,0,1,11,0,22,12,0.0
7,0,6,0.0,0.0,0.0,3,5,2.0,0,1252.0,...,0,356,1781,0,3,11,2,12,12,0.0
8,0,6,0.0,0.0,0.0,3,5,2.0,0,1252.0,...,0,356,1770,0,3,11,2,12,26,0.0
9,0,6,0.0,0.0,0.0,2,3,2.0,0,1252.0,...,0,356,1795,0,3,11,2,28,12,0.0


In [69]:
train = pd.merge(train, prop, on='parcelid', how='left')
print(train.head())
print('---------------------')
print(train.shape)

   parcelid  logerror transactiondate  transaction_month  \
0  14297519  0.025595      2017-01-01                  1   
1  17052889  0.055619      2017-01-01                  1   
2  14186244  0.005383      2017-01-01                  1   
3  12177905 -0.103410      2017-01-01                  1   
4  10887214  0.006940      2017-01-01                  1   

   airconditioningtypeid  architecturalstyletypeid  basementsqft  bathroomcnt  \
0                      0                         6           0.0          3.5   
1                      0                         6           0.0          1.0   
2                      0                         6           0.0          2.0   
3                      0                         6           0.0          3.0   
4                      0                         6           0.0          3.0   

   bedroomcnt  buildingclasstypeid           ...             raw_census_1  \
0         4.0                    3           ...                        1  

In [70]:
log_errors = train['logerror']
train = train[train.logerror < np.percentile(log_errors, 99.5)]
train = train[train.logerror > np.percentile(log_errors, 0.5)]

print('upper limit: ', np.percentile(log_errors, 99.5))
print('lower limit: ', np.percentile(log_errors, 0.5))

upper limit:  0.8004549760705607
lower limit:  -0.46763610747284


In [71]:
x_train = train.drop(['parcelid', 'logerror','transactiondate'], axis=1)
y_train = train['logerror']

In [72]:
params = {
    'eta': 0.02,
    'max_depth': 1,
    'subsample': 0.90,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': np.mean(y_train),
    'silent': 1
}

In [73]:
dtrain = xgb.DMatrix(x_train, y_train)

In [74]:
dtrain

In [75]:
cv_result = xgb.cv(params, 
                   dtrain, 
                   nfold=5,
                   num_boost_round=200,
                   early_stopping_rounds=5,
                   verbose_eval=10, 
                   show_stdv=False
                  )
num_boost_rounds = len(cv_result)
print(num_boost_rounds)

[0]	train-mae:0.0587804	test-mae:0.0587812
[10]	train-mae:0.0587358	test-mae:0.0587412
[20]	train-mae:0.058703	test-mae:0.0587124
[30]	train-mae:0.0586774	test-mae:0.0586914
[40]	train-mae:0.0586566	test-mae:0.0586738
[50]	train-mae:0.0586384	test-mae:0.0586578
[60]	train-mae:0.058622	test-mae:0.0586444
[70]	train-mae:0.0586078	test-mae:0.0586328
[80]	train-mae:0.0585952	test-mae:0.0586246
[90]	train-mae:0.0585848	test-mae:0.0586178
[100]	train-mae:0.0585722	test-mae:0.058607
[110]	train-mae:0.0585602	test-mae:0.0585974
[120]	train-mae:0.0585482	test-mae:0.0585872
[130]	train-mae:0.058537	test-mae:0.058577
[140]	train-mae:0.0585274	test-mae:0.0585692
[150]	train-mae:0.05852	test-mae:0.0585624
[160]	train-mae:0.0585124	test-mae:0.0585548
[170]	train-mae:0.0585058	test-mae:0.0585508
[180]	train-mae:0.0584976	test-mae:0.0585436
[190]	train-mae:0.0584904	test-mae:0.0585374
[199]	train-mae:0.0584836	test-mae:0.0585336
200


In [76]:
mdl = xgb.train(params, dtrain, num_boost_round=num_boost_rounds)

In [77]:
oct_test = test.copy()

In [78]:
oct_test['transaction_month'] = np.repeat(10, oct_test.shape[0])

In [79]:
def reorder(df):
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df.loc[:, cols]
    return df

In [80]:
oct_test = reorder(oct_test)

In [81]:
oct_test

,transaction_month,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,raw_census_1,census_2,raw_census_2,census_3,raw_census_3,census_4,raw_census_4,propertyzoningdesc_1,propertyzoningdesc_3,taxdelinquencyyear_past
0,10,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,2892,0,8,11,7,22,26,0.0
1,10,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,2888,0,8,11,7,22,10,0.0
2,10,0,6,0.0,0.0,0.0,4,5,2.0,0,...,0,356,2884,0,7,11,6,22,12,0.0
3,10,0,6,0.0,0.0,0.0,2,5,2.0,0,...,0,356,1073,0,1,11,0,22,12,0.0
4,10,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,916,0,1,11,0,22,21,0.0
5,10,0,6,0.0,0.0,0.0,3,3,2.0,0,...,0,356,947,0,1,11,0,22,12,0.0
6,10,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,1089,0,1,11,0,22,12,0.0
7,10,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,1781,0,3,11,2,12,12,0.0
8,10,0,6,0.0,0.0,0.0,3,5,2.0,0,...,0,356,1770,0,3,11,2,12,26,0.0
9,10,0,6,0.0,0.0,0.0,2,3,2.0,0,...,0,356,1795,0,3,11,2,28,12,0.0


In [82]:
d_oct_test = xgb.DMatrix(oct_test)

In [83]:
d_oct_test

In [84]:
oct_pred = mdl.predict(d_oct_test)

In [85]:
ss = pd.read_csv('data/sample_submission.csv')

In [86]:
sub_oct_pred = []
sub_nov_pred = []
sub_dec_pred = []

for i,predict in enumerate(oct_pred):
    sub_oct_pred.append(str(round(predict,4)))
sub_oct_pred=np.array(sub_oct_pred)

In [87]:
nov_test = test.copy()

In [88]:
nov_test['transaction_month'] = np.repeat(11, nov_test.shape[0])

In [89]:
nov_test = reorder(nov_test)

In [90]:
d_nov_test = xgb.DMatrix(nov_test)

In [91]:
nov_pred = mdl.predict(d_nov_test)

In [92]:
for i,predict in enumerate(nov_pred):
    sub_nov_pred.append(str(round(predict,4)))
sub_nov_pred=np.array(sub_nov_pred)

In [93]:
dec_test = test.copy()

In [94]:
dec_test['transaction_month'] = np.repeat(12, dec_test.shape[0])

In [95]:
dec_test = reorder(dec_test)

In [96]:
d_dec_test = xgb.DMatrix(dec_test)

In [97]:
dec_pred = mdl.predict(d_dec_test)

In [98]:
for i,predict in enumerate(dec_pred):
    sub_dec_pred.append(str(round(predict,4)))
sub_dec_pred=np.array(sub_dec_pred)

In [99]:
sub2 = pd.DataFrame({'ParcelId': prop['parcelid'].astype(np.int32), '201710':sub_oct_pred, '201711': sub_nov_pred, '201712':sub_dec_pred})

In [108]:
del sub2['ParcelId']

In [109]:
sub2

,201710,201711,201712
0,0.0141,0.0141,0.0141
1,0.0116,0.0116,0.0116
2,0.0202,0.0202,0.0202
3,0.0202,0.0202,0.0202
4,0.0116,0.0116,0.0116
5,0.0116,0.0116,0.0116
6,0.0116,0.0116,0.0116
7,0.0116,0.0116,0.0116
8,0.0141,0.0141,0.0141
9,0.0116,0.0116,0.0116


In [101]:
sub

,ParcelId,201610,201611,201612
0,10754147,0.0073,0.0073,0.0073
1,10759547,0.0073,0.0073,0.0073
2,10843547,0.0189,0.0189,0.0189
3,10859147,0.0111,0.0111,0.0111
4,10879947,0.0073,0.0073,0.0073
5,10898347,0.0077,0.0077,0.0077
6,10933547,0.0062,0.0062,0.0062
7,10940747,0.01,0.01,0.01
8,10954547,0.0073,0.0073,0.0073
9,10976347,0.01,0.01,0.01


In [111]:
sub_result = pd.concat([sub, sub2], axis=1)
sub_result

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.0073,0.0073,0.0073,0.0141,0.0141,0.0141
1,10759547,0.0073,0.0073,0.0073,0.0116,0.0116,0.0116
2,10843547,0.0189,0.0189,0.0189,0.0202,0.0202,0.0202
3,10859147,0.0111,0.0111,0.0111,0.0202,0.0202,0.0202
4,10879947,0.0073,0.0073,0.0073,0.0116,0.0116,0.0116
5,10898347,0.0077,0.0077,0.0077,0.0116,0.0116,0.0116
6,10933547,0.0062,0.0062,0.0062,0.0116,0.0116,0.0116
7,10940747,0.01,0.01,0.01,0.0116,0.0116,0.0116
8,10954547,0.0073,0.0073,0.0073,0.0141,0.0141,0.0141
9,10976347,0.01,0.01,0.01,0.0116,0.0116,0.0116


In [112]:
sub_result.to_csv('submission.gz', index=False, compression = 'gzip')